In [1]:
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk


In [2]:
# load the matching document contents
df_text = pd.read_csv("data/trackGBV_xls_match.csv")
print(len(df_text))

809


In [3]:
df_labels = pd.read_csv("data/trackGBV_labels.csv")
print(len(df_labels))

809


## There are 5 types of document

* Judgment
* Sentence
* Ruling
* Decision
* Other

In [4]:
# Get the index of the document type
df_text["j_idx"]= df_text["contents"].str.find("JUDGMENT") 
df_text["s_idx"]= df_text["contents"].str.find("SENTENCE") 
df_text["r_idx"]= df_text["contents"].str.find("RULING")
df_text["d_idx"]= df_text["contents"].str.find("DECISION")
df_text["o_idx"] = df_text["contents"].str.find('APPLICATION FOR BAIL PENDING APPEAL')

In [5]:
# check if there are any duplicate document_types
# first j_idx and s_idx, then do the others

print("Number of docs that are:")
print()
print("JUDGMENT:", len(df_text.loc[(df_text.j_idx != -1)])) # 203
print("SENTENCE:", len(df_text.loc[(df_text.s_idx != -1)])) # 565
print("RULING:", len(df_text.loc[(df_text.r_idx != -1)]))   # 47
print("DECISION:", len(df_text.loc[(df_text.d_idx != -1)])) # 4
print("OTHER:", len(df_text.loc[(df_text.o_idx != -1)])) # 4

print("JUDGMENT & SENTENCE:", len(df_text.loc[((df_text.j_idx != -1) & (df_text.s_idx != -1))])) # 9
print("JUDGMENT & RULING:", len(df_text.loc[((df_text.j_idx != -1) & (df_text.r_idx != -1))]))   # 1
print("JUDGMENT & DECISION:", len(df_text.loc[((df_text.j_idx != -1) & (df_text.d_idx != -1))])) # 0

print("SENTENCE & RULING:", len(df_text.loc[((df_text.s_idx != -1) & (df_text.r_idx != -1))]))   # 2
print("SENTENCE & DECISION:", len(df_text.loc[((df_text.s_idx != -1) & (df_text.d_idx != -1))])) # 0

print("RULING & DECISION:", len(df_text.loc[((df_text.r_idx != -1) & (df_text.d_idx != -1))])) # 0

Number of docs that are:

JUDGMENT: 196
SENTENCE: 555
RULING: 46
DECISION: 4
OTHER: 1
JUDGMENT & SENTENCE: 9
JUDGMENT & RULING: 1
JUDGMENT & DECISION: 0
SENTENCE & RULING: 2
SENTENCE & DECISION: 0
RULING & DECISION: 0


In [6]:
# find and replace missing judgment statements
df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('J U D G M E N T')
            ), 'j_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('J U D G M E N T')
            )]['contents'].str.find('J U D G M E N T')

df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('JUDGEMENT')
            ), 'j_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('JUDGEMENT')
            )]['contents'].str.find('JUDGEMENT')

In [7]:
# find and replace missing ruling statements

df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('R U L I N G')
            ), 'r_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('R U L I N G')
            )]['contents'].str.find('R U L I N G')

In [8]:
# find and replace missing sentence statements

df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('S E N T E N C E')
            ), 's_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('S E N T E N C E')
            )]['contents'].str.find('S E N T E N C E')

In [9]:
# find and replace missing 'other' statements
df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('CHARGE')
            ), 'o_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('CHARGE')
            )]['contents'].str.find('CHARGE')

df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('PUNISHMENT')
            ), 'o_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('PUNISHMENT')
            )]['contents'].str.find('PUNISHMENT')

df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('ORDER OF SUMMARY DISMISSAL')
            ), 'o_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('ORDER OF SUMMARY DISMISSAL')
            )]['contents'].str.find('ORDER OF SUMMARY DISMISSAL')

In [10]:
# Fix the lower case 'sentence' and 'judgment' cases (there are only 6)
df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('Sentence')
            ), 's_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('Sentence')
            )]['contents'].str.find('Sentence')

df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('Judgment')
            ), 'j_idx' ] = df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & 
              df_text.contents.str.contains('Judgment')
            )]['contents'].str.find('Judgment')

In [11]:
# Check we have captured every doc type = there should be none
df_text.loc[((df_text.j_idx == -1) & (df_text.s_idx == -1) & (df_text.r_idx == -1) & (df_text.d_idx == -1) & (df_text.o_idx == -1))]

,docid,contents,j_idx,s_idx,r_idx,d_idx,o_idx


In [46]:
# Replace all -1s to a None so that we can get the "minimum Non -1 value to key off of in or replace text"
df_text.loc[df_text['s_idx'] == -1, 's_idx'] = None
df_text.loc[df_text['j_idx'] == -1, 'j_idx'] = None
df_text.loc[df_text['r_idx'] == -1, 'r_idx'] = None
df_text.loc[df_text['d_idx'] == -1, 'd_idx'] = None
df_text.loc[df_text['o_idx'] == -1, 'o_idx'] = None

In [47]:
df_text['type_idx'] = df_text[['s_idx', 'j_idx', 'r_idx', 'd_idx', 'o_idx']].min(axis=1).astype(int)

In [48]:
# Remove all the parts of the document up to the document type
for row in df_text.itertuples():
    idx = row.type_idx
    new_text = row.contents[idx:]
    df_text.at[row.Index, 'cleaned_contents'] = new_text

In [49]:
# Remove URL
df_text['cleaned_contents'] = df_text['cleaned_contents'].replace(r'http://www.paclii.org.*', '<URL>', regex=True)           

In [50]:
print(df_text['cleaned_contents'][0])

SENTENCE
[Child Rape]

[1] On the 11th October 2011 the accused entered a plea of guilty to one charge of rape contrary to Section 207(1) & (2) (b) of the Crimes Decree 2009, the particulars being that on the 14th March 2010 at Uciwai settlement, Korolevu he raped a child with his finger. He agreed to facts put to him, whereupon he was found guilty and convicted.

[2] The facts admitted by the accused were that in March 2010 he was staying with the victims' family, he being related to the victims' father. On the 14th March 2010, the mother had left home to visit the father in hospital. She left the victim, who at the time was 9 years old and her elder brother in the care of the accused.

[3] After dinner the two children went to sleep on a mattress. The boy asked the accused to sleep on the mattress with them as his mother has not yet returned. The accused blew out the candle and joined them on the mattress. The boy told the victim to lie in the middle between him and the accused. Whil

In [51]:
# replace all dates?
# replace all names?

In [67]:
df_text.to_csv(r'data/cleaned_text.csv', index = False)